In [167]:
import requests
from dotenv import load_dotenv
import os
import pandas as pd
from concurrent.futures import ThreadPoolExecutor
from tqdm import tqdm


In [128]:
# Load environment variables from the .env file
load_dotenv()

# Get the API token and database ID from environment variables
NOTION_API_TOKEN = os.getenv("NOTION_API_TOKEN")
NOTION_DATABASE_ID_DASHBOARD = os.getenv("NOTION_DATABASE_ID_DASHBOARD")
NOTION_DATABASE_ID_REF = os.getenv("NOTION_DATABASE_ID_REF")

# Set up the headers for the API request
headers = {
    "Authorization": f"Bearer {NOTION_API_TOKEN}",
    "Notion-Version": "2021-08-16",
    "Content-Type": "application/json",
}

In [182]:
class NotionAPI:
    def __init__(self, NOTION_API_TOKEN, NOTION_DATABASE_ID_DASHBOARD, NOTION_DATABASE_ID_REF):
        self.NOTION_API_TOKEN = NOTION_API_TOKEN
        self.NOTION_DATABASE_ID_DASHBOARD = NOTION_DATABASE_ID_DASHBOARD
        self.NOTION_DATABASE_ID_REF = NOTION_DATABASE_ID_REF
        self.headers = {
            "Notion-Version": "2022-06-28",
            "Accept": "application/json",
            "Content-Type": "application/json",
            "Authorization": f"Bearer {NOTION_API_TOKEN}",
        }
        self._get_ref_df()
        self._get_dashboard_df()

        
    def _get_ref_df(self):
        self.ref_df = self.get_df(self.NOTION_DATABASE_ID_REF)
        self.ref_df['activity'] = self.ref_df['properties.Productions.title'].apply(lambda t: t[0]['text']['content'])
        self.ref_df = self.ref_df[['id','activity','properties.P_PC.select.name','properties.difficulty1-5.number']]
        print('ref_df loaded')
        
    def _get_dashboard_df(self):
        self.dashboard_df = self.get_df(self.NOTION_DATABASE_ID_DASHBOARD)
        self.dashboard_df['created_time'] = pd.to_datetime(self.dashboard_df['created_time'])
        print('dashboard loaded')

    def get_df(self,NOTION_DATABASE_ID):
        payload = {}

        response = requests.post(
            f"https://api.notion.com/v1/databases/{NOTION_DATABASE_ID}/query",
            headers=self.headers,
            json=payload,
        )

        if response.status_code == 200:
            data = response.json()
            result_df = pd.json_normalize(data['results'])
            return result_df
        else:
            print(f"Failed to retrieve data. Status code: {response.status_code}")
            print(response.text)
            return None

    def get_page_data(self, page_id):
        response = requests.get(f"https://api.notion.com/v1/blocks/{page_id}/children", headers=self.headers)

        if response.status_code == 200:
            page_data = response.json()
            return page_data
        else:
            print(f"Failed to retrieve page data. Status code: {response.status_code}")
            print(response.text)
            return None
        
    def _get_daily_check(self,page_id,created_time):
        try:
            blocks = self.get_page_data(page_id)['results']
            daily_check = self.get_df(blocks[0]['id'])
            daily_check['created_time'] = created_time 
            daily_check['ref_id'] = daily_check['properties.Activity.relation'].apply(lambda x: x[0]['id'])
            return daily_check[['created_time','ref_id','properties.Yes.checkbox']]
        except Exception as e:
            print(e)
            return pd.DataFrame()
    
    def get_daily_checks(self, ids, created_times, workers=4):
        with ThreadPoolExecutor(max_workers=workers) as executor:
            results = list(tqdm(executor.map(self._get_daily_check, ids, created_times), total=len(ids)))
        return results

    def get_everything(self, workers=5):
        daily_checks = self.get_daily_checks(self.dashboard_df.id, self.dashboard_df.created_time, workers=workers)
        df_daily_checks =  pd.concat(daily_checks) 
        df_daily_checks = df_daily_checks.merge(
                        self.ref_df,
                        left_on=['ref_id'],
                        right_on=['id'] 
                    )
        return df_daily_checks
    

In [183]:
notion_api = NotionAPI(NOTION_API_TOKEN, NOTION_DATABASE_ID_DASHBOARD, NOTION_DATABASE_ID_REF)

ref_df loaded
dashboard loaded


In [184]:
df_all = notion_api.get_everything(10)

  0%|          | 0/56 [00:00<?, ?it/s]

  2%|▏         | 1/56 [00:00<00:29,  1.86it/s]

Failed to retrieve data. Status code: 404
{"object":"error","status":404,"code":"object_not_found","message":"Could not find database with ID: 4cc78746-d4a7-477c-854f-66c381cdbe12. Make sure the relevant pages and databases are shared with your integration."}
'NoneType' object does not support item assignment
Failed to retrieve data. Status code: 404
{"object":"error","status":404,"code":"object_not_found","message":"Could not find database with ID: f5a40925-cbab-4515-b468-af14d134b86b. Make sure the relevant pages and databases are shared with your integration."}
'NoneType' object does not support item assignment


100%|██████████| 56/56 [00:16<00:00,  3.36it/s]


# Analysis

In [187]:
df_all.head()

,created_time,ref_id,properties.Yes.checkbox,id,activity,properties.P_PC.select.name,properties.difficulty1-5.number
0,2023-09-17 11:00:00+00:00,29fcaf58-1b6b-4650-8229-ec43e791bf33,False,29fcaf58-1b6b-4650-8229-ec43e791bf33,DO: Reading,Production_Capacity,3
1,2023-09-16 11:00:00+00:00,29fcaf58-1b6b-4650-8229-ec43e791bf33,True,29fcaf58-1b6b-4650-8229-ec43e791bf33,DO: Reading,Production_Capacity,3
2,2023-09-15 11:00:00+00:00,29fcaf58-1b6b-4650-8229-ec43e791bf33,False,29fcaf58-1b6b-4650-8229-ec43e791bf33,DO: Reading,Production_Capacity,3
3,2023-09-14 11:00:00+00:00,29fcaf58-1b6b-4650-8229-ec43e791bf33,False,29fcaf58-1b6b-4650-8229-ec43e791bf33,DO: Reading,Production_Capacity,3
4,2023-09-13 11:00:00+00:00,29fcaf58-1b6b-4650-8229-ec43e791bf33,True,29fcaf58-1b6b-4650-8229-ec43e791bf33,DO: Reading,Production_Capacity,3


In [189]:
df_all.groupby('activity')['properties.Yes.checkbox'].sum().sort_values()

activity
DO: Meditation                   8
DO: Wake Early                  10
DO: Writing/Reflecting          11
DONT: Stay up late (11Pm)       18
DO: Reading                     22
DO: Cleaning up living space    24
DO: Running/Biking              29
DONT: Leave a mess              39
DONT: Binge Playing Games       47
DONT: Binge Watching TVs        48
Name: properties.Yes.checkbox, dtype: int64